In [350]:
%matplotlib inline
import numpy as np
import scipy as sp
import time
import matplotlib.pyplot as plt
import random
from collections import namedtuple, defaultdict, Counter
import math
import pickle

# Data pull

In [330]:
with open('raw_data.txt') as f:
    lines = f.readlines()
lines = [x.strip().split('\t') for x in lines]

name_to_cid = {}
cid_to_name = {}

for resume in lines:
    for name in resume:
        if name not in name_to_cid:
            name_to_cid[name] = len(name_to_cid)
            cid_to_name[name_to_cid[name]] = name
            
G = [[name_to_cid[name] for name in reversed(row)] for row in lines]
num_students = len(G)
num_companies = len(name_to_cid)
company_counter = Counter([cid for row in G for cid in row])

cid_to_rows = defaultdict(set)
for i, row in enumerate(G):
    for cid in row:
        cid_to_rows[cid].add(i)
        
num_students, num_companies, G[0], company_counter[name_to_cid['Google']], len(cid_to_rows[name_to_cid['Google']])

(1350, 2370, [3, 2, 1, 0], 101, 72)

# Ranking

In [331]:
def _dcg(r, variant):
    # O(R)
    p = r.size
    if variant == 1:
        return r[0] + np.sum(r[1:] / np.log2(np.arange(2, p + 1)))
    else:
        return np.sum((np.exp2(r) - 1.0) / np.log2(np.arange(1, p + 1) + 1.0))

def _ndcg(r, variant):
    # O(R log R)
    sorted_r = np.sort(r)[::-1]
    return _dcg(r, variant) / _dcg(sorted_r, variant)
    
def ndcg(relevances, variant=2):
    assert variant in [1, 2]
    z = {}
    val = 1
    for r in sorted(relevances):
        if r not in z:
            z[r] = val
            val += 1
    ans = [z[r] for r in relevances]
    #print(ans)
    return _ndcg(np.asfarray(ans), variant)

ndcg([100.0, 200.0, 100.0]), ndcg([200.0, 100.0, 200.0])

(0.82131371461378277, 0.9514426589871553)

In [448]:
def get_relevance(ranking):
    z = {}
    for i, cid in enumerate(ranking):
        z[cid] = num_companies - i
    return z

def score(ranking):
    relevance = get_relevance(ranking)
    _score = sum(ndcg([relevance[cid] for cid in row]) for row in G)
    return _score

def get_partial_relevance(cid_to_rank, cid):
    z = {}
    for row_num in cid_to_rows[cid]:
        for company in G[row_num]:
            if company not in z:
                z[company] = num_companies - cid_to_rank[company]
    return z

def get_partial_score(cid_to_rank, cid):
    relevance = get_partial_relevance(cid_to_rank, cid)
    _score = sum(ndcg([relevance[cid] for cid in G[row_num]]) for row_num in cid_to_rows[cid])
    return _score

In [438]:
#ranking in descending order of goodness
#ranking = list(range(num_companies))
#np.random.shuffle(ranking)

In [490]:
current_score = score(ranking)
current_score

1329.4640746841908

In [491]:
def get_names(ranking):
    return [cid_to_name[cid] for cid in ranking]
get_names(filter(lambda x: company_counter[x] > 10, ranking))[:10], get_names(ranking)[:10]
#ranking[519], ranking[0] = ranking[0], ranking[519]
#ranking.index(name_to_cid['Google'])

(['Palantir Technologies',
  'Twitter',
  'Google',
  'ContextLogic',
  'Microsoft',
  'Facebook',
  'Yelp',
  'Arista Networks, Inc.',
  'Amazon',
  'Bloomberg'],
 ['Vine',
  'UCIC',
  'LightBot Inc',
  'uwflow.com',
  'Quora',
  'Kite',
  'Tinfoil Security',
  'Palantir Technologies',
  'Armor Games Inc',
  'Bridgit'])

In [487]:
def insert_ranking_test(from_val, to_val, ranking):
    assert from_val != to_val
    ranking_test = ranking[:]
    v = ranking_test.pop(from_val)
    ranking_test.insert(to_val, v)
    assert len(set(ranking_test)) == num_companies
#    assert len(ranking_test) == num_companies
    return ranking_test

def get_ranking_map(ranking):
    z = {}
    for i, v in enumerate(ranking):
        z[v] = i
    return z

def fast_insert_score(from_val, to_val, cid, ranking_map):
    assert ranking_map[cid] == from_val
    assert from_val != to_val
    old_partial_score = get_partial_score(ranking_map, cid)
    if to_val < from_val:
        ranking_map[cid] = to_val - 0.5
    else:
        ranking_map[cid] = to_val + 0.5
    new_partial_score = get_partial_score(ranking_map, cid)
    ranking_map[cid] = from_val
    return new_partial_score - old_partial_score
    
def bsearch_low(x, ranking, current_score):
    ranking_map = get_ranking_map(ranking)
    y_low = 0 #insert before this guy
    y_high = x # non-inclusive

    while y_high - y_low > 1:
        y_mid = (y_low + y_high - 1) // 2 # this rounds closer to y_low
        y_mid = random.randint(y_low, y_mid) # more aggressive, more likely to overshoot

        fast_score = fast_insert_score(x, y_mid, ranking[x], ranking_map)

        if math.isclose(fast_score, 0.0, abs_tol=1e-5):
            y_high = y_mid + 1
        elif fast_score > 0.0:
            ranking_test = insert_ranking_test(x, y_mid, ranking)
            new_score = score(ranking_test)
            assert new_score > current_score
            current_score = new_score
            ranking = ranking_test
            return ranking, current_score
        else:
            y_low = y_mid + 1
            
    if y_low != x:
        ranking_test = insert_ranking_test(x, y_low, ranking)
        new_score = score(ranking_test)
        if new_score > current_score:
            current_score = new_score
            ranking = ranking_test
    return ranking, current_score

def bsearch_high(x, ranking, current_score):
    ranking_map = get_ranking_map(ranking)
    y_low = x #non-inclusive
    y_high = num_companies - 1 #insert after this guy

    while y_high - y_low > 1:
        y_mid = (y_low + y_high + 2) // 2 # this rounds closer to y_high
        y_mid = random.randint(y_mid, y_high) # more aggressive, more likely to overshoot

        fast_score = fast_insert_score(x, y_mid, ranking[x], ranking_map)

        if math.isclose(fast_score, 0.0, abs_tol=1e-5):
            y_low = y_mid - 1
        elif fast_score > 0.0:
            ranking_test = insert_ranking_test(x, y_mid, ranking)
            new_score = score(ranking_test)
            assert new_score > current_score
            current_score = new_score
            ranking = ranking_test
            return ranking, current_score
        else:
            y_high = y_mid - 1
    
    if y_high != x:
        ranking_test = insert_ranking_test(x, y_high, ranking)
        new_score = score(ranking_test)
        if new_score > current_score:
            current_score = new_score
            ranking = ranking_test
    return ranking, current_score


current_score = score(ranking)
    tmp_ranking = ranking[:]
    tmp_score = current_score
for _ in range(100):
    while True: #hyper
        prev_tmp_score = tmp_score
        print("Iteration:", _, tmp_score, time.strftime("%Y-%m-%d %H:%M:%S"))

        for x in range(0, num_companies): #hyper
            if x % 200 == 0:
                print('x =', x, num_companies, tmp_score, time.strftime("%Y-%m-%d %H:%M:%S"))
            tmp_ranking, tmp_score = bsearch_low(x, tmp_ranking, tmp_score)
            tmp_ranking, tmp_score = bsearch_high(x, tmp_ranking, tmp_score)
            if tmp_score > current_score:
                current_score = tmp_score
                ranking = tmp_ranking
                print("YAY UPDATED: ", current_score, get_names(ranking)[:20])

        if math.isclose(tmp_score, prev_tmp_score, abs_tol=1e-5):
            break

Iteration: 0 1329.38500662 2016-02-28 15:38:09
x = 0 2370 1329.38500662 2016-02-28 15:38:09
x = 200 2370 1329.38500662 2016-02-28 15:38:36
x = 400 2370 1329.38500662 2016-02-28 15:39:02
x = 600 2370 1329.38500662 2016-02-28 15:39:25
x = 800 2370 1329.38500662 2016-02-28 15:39:48
x = 1000 2370 1329.38500662 2016-02-28 15:40:13
x = 1200 2370 1329.38500662 2016-02-28 15:40:36
x = 1400 2370 1329.38500662 2016-02-28 15:40:59
x = 1600 2370 1329.38500662 2016-02-28 15:41:22
x = 1800 2370 1329.38500662 2016-02-28 15:41:44
x = 2000 2370 1329.38500662 2016-02-28 15:42:07
x = 2200 2370 1329.38500662 2016-02-28 15:42:29
Iteration: 1 1329.38500662 2016-02-28 15:42:48
x = 0 2370 1329.38500662 2016-02-28 15:42:48
x = 200 2370 1329.38500662 2016-02-28 15:43:12
x = 400 2370 1329.38500662 2016-02-28 15:43:36
x = 600 2370 1329.38500662 2016-02-28 15:43:59
x = 800 2370 1329.38500662 2016-02-28 15:44:22
x = 1000 2370 1329.38500662 2016-02-28 15:44:47
x = 1200 2370 1329.38500662 2016-02-28 15:45:10
x = 1400

KeyboardInterrupt: 

In [492]:
pickle.dump(ranking, open('final_ranking.txt', 'wb'))